In [22]:
import pandas as pd
import numpy as np

In [59]:
def compute_mean_sector_multiples():
    
    # Load databases
    general_info = pd.read_csv('./databases/general_info.csv')
    last_valuations = pd.read_csv('./databases/last_valuations.csv')
    
    # Select relevant data and merge
    general_info = general_info[['ticker', 'shortName', 'sector', 'industry']]
    last_valuations = pd.merge(general_info, last_valuations, on='ticker')
    last_valuations = last_valuations[['sector', 'priceToBook', 'enterpriseToRevenue', 'enterpriseToEbitda', 'forwardPE', 'trailingPE', 'pegRatio', 'priceToSalesTrailing12Months']]
    
    # Group by sector for test purposes but probably better by industry with a bigger sample
    mean_sector_multiples = multiples.groupby(by='sector').mean()
    
    return mean_sector_multiples

In [124]:
def reformat_valuation():
    
    # Quick&dirty, review databases architecture to have EV, bridge, VT, shares, stock price
    # Check diff stock_price computation with regularMarketPrice (probably fully diluted effect)
    last_valuations = pd.read_csv('./databases/last_valuations.csv')
    stock_info = pd.read_csv('./databases/stock_info.csv')
    
    # Select
    last_valuations = last_valuations[['ticker', 'bookValue', 'enterpriseValue', 'marketCap']]
    stock_info = stock_info[['ticker', 'sharesOutstanding']]
    
    # Join
    last_valuations = pd.merge(last_valuations, stock_info, on='ticker')
    
    # Transform
    last_valuations['bridge_enterpriseValue_marketCap'] = last_valuations['enterpriseValue'] - last_valuations['marketCap']
    last_valuations['stock_price'] = last_valuations['marketCap'] / last_valuations['sharesOutstanding']
    
    # Arrange
    last_valuations = last_valuations[['ticker', 'bookValue', 'enterpriseValue', 'bridge_enterpriseValue_marketCap', 'marketCap', 'sharesOutstanding', 'stock_price']]
    
    return last_valuations

In [115]:
def revenue_valuation(enterpriseToRevenue, totalRevenue, bridge_enterpriseValue_marketCap):
    
    enterpriseValue = enterpriseToRevenue * totalRevenue
    marketCap = enterpriseValue - bridge_enterpriseValue_marketCap
    
    return marketCap

In [116]:
def ebitda_valuation(enterpriseToEbitda, ebitda, bridge_enterpriseValue_marketCap):
    
    enterpriseValue = enterpriseToEbitda * ebitda
    marketCap = enterpriseValue - bridge_enterpriseValue_marketCap
    
    return marketCap

In [117]:
def earnings_valuation(trailingPE, earnings, bridge_enterpriseValue_marketCap):
    
    marketCap = trailingPE * earnings
    enterpriseValue = marketCap + bridge_enterpriseValue_marketCap
    
    return marketCap   

In [118]:
def book_valuation(priceToBook, bookValue, bridge_enterpriseValue_marketCap):
    
    marketCap = priceToBook * bookValue
    enterpriseValue = marketCap + bridge_enterpriseValue_marketCap
    
    return marketCap

In [148]:
def peers_valuation():

    # Load
    financials = pd.read_csv('./databases/financials.csv')
    general_info = pd.read_csv('./databases/general_info.csv')
    mean_sector_multiples = compute_mean_sector_multiples()
    last_valuations = reformat_valuation()

    # Select
    financials = financials[['ticker', 'totalRevenue', 'ebitda', 'freeCashflow']] # Missing earnings for now, used FCF instead just for computation mecanism (but result will be meaningless)
    general_info = general_info[['ticker', 'sector']]
    last_valuations = last_valuations[['ticker', 'bookValue', 'bridge_enterpriseValue_marketCap']]

    # Join
    financials = pd.merge(general_info, financials, on='ticker')
    financials = pd.merge(financials, last_valuations, on='ticker')
    valuation = pd.merge(financials, mean_sector_multiples, on='sector')

    # Apply valuation functions
    valuation['marketCap_revenue'] = valuation.apply(lambda x: revenue_valuation(x['enterpriseToRevenue'], x['totalRevenue'], x['bridge_enterpriseValue_marketCap']), axis=1)
    valuation['marketCap_ebitda'] = valuation.apply(lambda x: ebitda_valuation(x['enterpriseToEbitda'], x['ebitda'], x['bridge_enterpriseValue_marketCap']), axis=1)
    valuation['stock_price_book'] = valuation.apply(lambda x: book_valuation(x['priceToBook'], x['bookValue'], x['bridge_enterpriseValue_marketCap']), axis=1)

    # TBU with earnings and not freeCashflow !!!! ---------------------
    valuation['marketCap_earnings'] = valuation.apply(lambda x: earnings_valuation(x['trailingPE'], x['freeCashflow'], x['bridge_enterpriseValue_marketCap']), axis=1)

    # Final select
    valuation = valuation[['ticker', 'marketCap_revenue', 'marketCap_ebitda', 'marketCap_earnings', 'stock_price_book']]
    
    return valuation

In [170]:
def relative_std(values):
    
    std = np.nanstd(values)
    mean = np.nanmean(values)
    
    relative_std = (std / mean)
    
    return relative_std

In [180]:
def to_stock_price():
    
    # Load
    valuation = peers_valuation()
    stock_info = pd.read_csv('./databases/stock_info.csv')
    
    # Select
    stock_info = stock_info[['ticker', 'sharesOutstanding']]

    # Join
    valuation = pd.merge(valuation, stock_info, on='ticker')

    # Transform
    valuation['stock_price_revenue'] = valuation['marketCap_revenue'] / valuation['sharesOutstanding']
    valuation['stock_price_ebitda'] = valuation['marketCap_ebitda'] / valuation['sharesOutstanding']
    valuation['stock_price_earnings'] = valuation['marketCap_earnings'] / valuation['sharesOutstanding']
    
    valuation['mean_stock_price'] = valuation.apply(lambda x: np.nanmean([x['stock_price_book'], x['stock_price_revenue'], x['stock_price_ebitda'], x['stock_price_earnings']]), axis=1)
    valuation['relative_std_stock_price'] = valuation.apply(lambda x: relative_std([x['stock_price_book'], x['stock_price_revenue'], x['stock_price_ebitda'], x['stock_price_earnings']]), axis=1)
    
    # Final select
    valuation = valuation[['ticker', 'stock_price_book', 'stock_price_revenue', 'stock_price_ebitda', 'stock_price_earnings', 'mean_stock_price', 'relative_std_stock_price']]
    
    return valuation

In [174]:
def target_confidence(relative_std_stock_price):
    """Define a confidence level on the target price --> rule based on dispersion between different prediction methods"""
    
    if relative_std_stock_price < 0: # Probably an error in valuation if stock price is negative
        confidence = "Low"
    
    if relative_std_stock_price <= 0.25:
        confidence = "High"
    
    elif relative_std_stock_price <= 0.5:
        confidence = "Medium"
    
    else:
        confidence = "Low"
        
    return confidence

In [176]:
def price_differential():
    
    # Load databases
    valuation = to_stock_price()
    last_prices = pd.read_csv('./databases/last_prices.csv')
    general_info = pd.read_csv('./databases/general_info.csv')
    
    # Select relevant data
    valuation = valuation[['ticker', 'mean_stock_price', 'relative_std_stock_price']]
    regularMarketPrice = last_prices[['ticker', 'regularMarketPrice']]
    general_info = general_info[['ticker', 'shortName']]
    
    # Join dfs
    differential_df = pd.merge(general_info, valuation, on='ticker')
    differential_df = pd.merge(differential_df, regularMarketPrice, on='ticker')
    
    # Compute diff
    differential_df['absolute_diff'] = differential_df['mean_stock_price'] - differential_df['regularMarketPrice']
    differential_df['relative_diff'] = differential_df['absolute_diff'] / differential_df['regularMarketPrice']
    
    # Set a confidence level
    differential_df['confidence'] = differential_df['relative_std_stock_price'].apply(lambda x: target_confidence(x))
    
    # Sort from most undervalued to most overvalued
    differential_df = differential_df.sort_values(by='relative_diff', ascending=False)
    
    return differential_df

In [181]:
price_differential()

,ticker,shortName,mean_stock_price,relative_std_stock_price,regularMarketPrice,absolute_diff,relative_diff,confidence
37,STLAP.PA,STELLANTIS NV,157.573119,0.643593,16.080,141.493119,8.799323,Low
16,GLE.PA,SOCIETE GENERALE,196.967825,0.698391,23.800,173.167825,7.275959,Low
27,RNO.PA,RENAULT,307.963901,1.296366,38.580,269.383901,6.982475,Low
6,CA.PA,CARREFOUR,112.137017,1.205797,17.355,94.782017,5.461367,Low
21,MT.AS,ARCELORMITTAL SA,123.563120,0.160737,24.930,98.633120,3.956403,High
36,WLN.PA,WORLDLINE,138.797143,0.878733,33.500,105.297143,3.143198,Low
13,EN.PA,BOUYGUES,121.909352,0.276648,30.760,91.149352,2.963243,Medium
5,BNP.PA,BNP PARIBAS ACT.A,172.157439,0.585241,57.720,114.437439,1.982631,Low
0,ACA.PA,CREDIT AGRICOLE,31.331015,0.571328,10.872,20.459015,1.881808,Low
30,SGO.PA,SAINT GOBAIN,136.164442,0.063485,55.730,80.434442,1.443288,High
